<a href="https://colab.research.google.com/github/aymenchibouti/doctorat/blob/main/lime_shap_gpt_res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=a4f384d9494633ecd919eb9a665f5eceb7fdf142dbdec53135f8bb0fbc4f32ed
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime


In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import shap
import lime
import lime.lime_tabular
import xgboost as xgb
import matplotlib.pyplot as plt

# Load the dataset
file_path = 'model1_210_features.csv'
data = pd.read_csv(file_path)

# Drop non-numeric columns that are not useful for prediction
data = data.drop(columns=['username', 'course_id', 'enrollment_id'])

# Handle missing values (fill with 0 or use mean/median imputation as necessary)
data.fillna(0, inplace=True)

# Separate features and target variable
X = data.drop(columns=['dropout'])  # Features
y = data['dropout']  # Target variable

# Standardize the features (important for models like Logistic Regression and XGBoost)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train models
# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# XGBoost
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Predictions and evaluation
models = [lr, rf, xgb_model]
model_names = ['Logistic Regression', 'Random Forest', 'XGBoost']

for model, name in zip(models, model_names):
    y_pred = model.predict(X_test)
    print(f"Evaluation for {name}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
    print("-" * 50)

# Explainability using SHAP (SHAP can work with tree-based models like RandomForest and XGBoost)
explainer_rf = shap.TreeExplainer(rf)
shap_values_rf = explainer_rf.shap_values(X_test)

# SHAP summary plot for Random Forest
shap.summary_plot(shap_values_rf, X_test, feature_names=X.columns)

# LIME - Local Interpretable Model-Agnostic Explanations
explainer_lime = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train,
    training_labels=y_train,
    mode="classification",
    feature_names=X.columns,
    class_names=["No Dropout", "Dropout"],
    verbose=True,
    random_state=42
)

# Pick a single instance for LIME explanation
instance = X_test[0]  # First instance in the test set
explanation_lime = explainer_lime.explain_instance(instance, xgb_model.predict_proba)

# Visualize LIME explanation
explanation_lime.show_in_notebook()

# Feature Importance (using Random Forest)
feature_importance = rf.feature_importances_
# Visualize the feature importance
plt.figure(figsize=(10, 8))
plt.barh(X.columns, feature_importance)
plt.title("Feature Importance (Random Forest)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()


Evaluation for Logistic Regression:
Accuracy: 0.8579
Precision: 0.8669
Recall: 0.9690
F1 Score: 0.9151
--------------------------------------------------
Evaluation for Random Forest:
Accuracy: 0.8595
Precision: 0.8870
Recall: 0.9422
F1 Score: 0.9138
--------------------------------------------------
Evaluation for XGBoost:
Accuracy: 0.8629
Precision: 0.8814
Recall: 0.9550
F1 Score: 0.9167
--------------------------------------------------
